## A notebook to get any (single) input field besides curlTau

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
#from xgcm import Grid
import requests
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import glob, os

import dask.array as da
from dask.diagnostics import ProgressBar

In [2]:
experiment = 'ssp585'
var = 'vmo_2d'

In [3]:
if experiment == 'control':
    path='/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20221223/CM4_piControl_c192_OM4p25_v8/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_monthly/ts/monthly/5yr/'
    ds_grid=xr.open_dataset('/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20221223/CM4_piControl_c192_OM4p25_v8/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_monthly/ocean_monthly.static.nc')
elif experiment == 'historical':
    path='/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20221223/CM4_historical_c192_OM4p25/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_monthly/ts/monthly/5yr/'
    ds_grid=xr.open_dataset('/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20221223/CM4_historical_c192_OM4p25/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_monthly/ocean_monthly.static.nc')
elif experiment == 'ssp585':
    path='/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20221223/CM4_ssp585_c192_OM4p25/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_monthly/ts/monthly/5yr/'
    ds_grid=xr.open_dataset('/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20221223/CM4_ssp585_c192_OM4p25/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_monthly/ocean_monthly.static.nc')

dsShortGrid=ds_grid.isel(xq=np.arange(1,1441)).isel(yq=np.arange(1,1081))

In [4]:
test=glob.glob(path + 'ocean_monthly.*.intz_rvxv_2d.nc')

if experiment == 'control':
    year_name_start_idx, year_name_end_idx = 161, 174
elif experiment == 'historical':
    year_name_start_idx, year_name_end_idx = 159, 172
elif experiment == 'ssp585':
    year_name_start_idx, year_name_end_idx = 155, 168

one=test[0]
times=[one[year_name_start_idx:year_name_end_idx]]

for tt in np.arange(1,len(test)): 
    one=test[tt]
    times=times+[one[year_name_start_idx:year_name_end_idx]]
    
times = sorted(times)
    
if experiment == 'control':
    pass
elif experiment == 'historical':
    times = times[-5:]
elif experiment == 'ssp585':
    #times = times[-4:]
    times = times[-12:-8]

times

['204001-204412', '204501-204912', '205001-205412', '205501-205912']

In [5]:
datasets = []

if experiment == 'control':
    pass
elif experiment == 'historical':
    start_yr, end_yr = '1992', '2013'
elif experiment == 'ssp585':
    pass

files = [f'{path}/ocean_monthly.{time}.{var}.nc' for time in times]
    
print('Gathering dataset...')
ds = xr.open_mfdataset(files, chunks={"time": 12}, parallel=True)
with ProgressBar():
    ds = ds.compute()
    
datasets.append(ds)
    
#datasets.append(ds_grid)

print('Merging datasets...')
if experiment == 'control':
    full_ds = xr.merge(datasets).compute()
elif experiment == 'historical':
    full_ds = xr.merge(datasets).sel(time=slice(start_yr, end_yr)).compute()
elif experiment == 'ssp585':
    full_ds = xr.merge(datasets).compute()

full_ds

Gathering dataset...
[########################################] | 100% Completed | 16.63 ss
Merging datasets...


<xarray.Dataset>
Dimensions:     (time: 240, nv: 2, yq: 1081, xh: 1440)
Coordinates:
  * nv          (nv) float64 1.0 2.0
  * time        (time) object 2040-01-16 12:00:00 ... 2059-12-16 12:00:00
  * xh          (xh) float64 -299.7 -299.5 -299.2 -299.0 ... 59.53 59.78 60.03
  * yq          (yq) float64 -80.43 -80.35 -80.27 -80.19 ... 89.78 89.89 90.0
Data variables:
    average_DT  (time) timedelta64[ns] 31 days 28 days ... 30 days 31 days
    average_T1  (time) datetime64[ns] 2039-11-16 2039-12-17 ... 2059-10-11
    average_T2  (time) datetime64[ns] 2039-12-17 2040-01-14 ... 2059-11-11
    time_bnds   (time, nv) object 2040-01-01 00:00:00 ... 2060-01-01 00:00:00
    vmo_2d      (time, yq, xh) float64 nan nan nan nan nan ... nan nan nan nan
Attributes:
    filename:          ocean_monthly.204001-204412.vmo_2d.nc
    title:             CM4_ssp585_c192_OM4p25
    associated_files:  areacello: 20400101.ocean_static.nc
    grid_type:         regular
    grid_tile:         N/A

In [6]:
full_ds = full_ds.mean('time').drop(['nv', 'average_DT'])
full_ds

<xarray.Dataset>
Dimensions:  (yq: 1081, xh: 1440)
Coordinates:
  * xh       (xh) float64 -299.7 -299.5 -299.2 -299.0 ... 59.53 59.78 60.03
  * yq       (yq) float64 -80.43 -80.35 -80.27 -80.19 ... 89.68 89.78 89.89 90.0
Data variables:
    vmo_2d   (yq, xh) float64 nan nan nan nan nan nan ... nan nan nan nan nan

In [7]:
full_ds.to_netcdf(f'Data_Step2/{experiment}_2040_2059/{var}.nc')